In [3]:
"""
train_cldnn_fixed.py
修复样本权重维度问题的CLDNN干扰分类模型训练
"""
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, 
                                   Dense, Dropout, BatchNormalization,
                                   Reshape, LSTM)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, 
                                      ModelCheckpoint)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# ---------- GPU配置 ----------
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 干扰类型定义 ----------
INTERFERENCE_TYPES = [
    "satellite_signal",  # 0: 无干扰
    "single_tone",       # 1
    "comb_spectra",      # 2
    "sweeping",          # 3
    "pulse",             # 4
    "frequency_hopping", # 5
    "same_frequency",    # 6
    "noise_fm",          # 7
    "noise_am",          # 8
    "random_combination" # 9
]
NUM_CLASSES = len(INTERFERENCE_TYPES)

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载并处理数据集"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "same_frequency": "Same_Frequency",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
        print("⚠️ 未找到干扰类型名称，使用默认值")
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
    else:
        type2label = {name: i for i, name in enumerate(INTERFERENCE_TYPES)}
    
    return {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "type2label": type2label,
        "label2name": {i: type_names[k] for k, i in type2label.items()},
        "L": int(data["L"])
    }

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据（复信号构造）"""
    # 信号标准化并转换为float32
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]], dtype=np.float32)
    
    # 数据集分割（保持分层抽样）
    X_train, X_test, y_train, y_test = train_test_split(
        signals, dataset["labels"],
        test_size=0.3, random_state=42, 
        stratify=dataset["labels"]
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test,
        test_size=0.5, random_state=42,
        stratify=y_test
    )
    
    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_train": y_train, "y_val": y_val, "y_test": y_test,
        "label2name": dataset["label2name"],
        "L": dataset["L"]
    }

# ---------- CLDNN模型构建 ----------
def build_cldnn(input_shape, num_classes):
    """构建CLDNN分类模型"""
    inputs = Input(shape=input_shape)
    x = Reshape((input_shape[0], 1))(inputs)  # [batch, length, 1]
    
    # CNN特征提取
    x = Conv1D(32, 7, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(64, 5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    # LSTM时序建模
    x = LSTM(64, return_sequences=True)(x)
    x = LSTM(32)(x)
    
    # 分类头
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(1e-3),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    return model

# ---------- 数据增强 ----------
@tf.function
def aug_fn(x):
    """信号增强（确保数据类型和维度正确）"""
    x = tf.cast(x, tf.float32)
    
    # 噪声增强
    if tf.random.uniform([], dtype=tf.float32) > 0.2:
        snr = tf.random.uniform([], 5., 25., dtype=tf.float32)
        noise = tf.random.normal(tf.shape(x), dtype=tf.float32) * tf.math.reduce_std(x) * 10.0**(-snr/20.0)
        x = x + noise
    
    # 位移增强
    if tf.random.uniform([], dtype=tf.float32) > 0.3:
        shift = tf.random.uniform([], -100, 100, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=0)
    
    # 缩放增强
    if tf.random.uniform([], dtype=tf.float32) > 0.3:
        scale = tf.random.uniform([], 0.7, 1.3, dtype=tf.float32)
        x = x * scale
    
    return x

# ---------- 训练函数 ----------
# 修改 train_single_model 函数中的数据集创建部分
def train_single_model(data, model_idx=0, epochs=120, batch=128):
    """训练单个CLDNN模型"""
    model = build_cldnn((data["L"],), len(data["label2name"]))
    
    # 类别权重（处理不平衡）
    cls_weights = compute_class_weight('balanced', 
                                     classes=np.unique(data['y_train']), 
                                     y=data['y_train'])
    sample_weights = np.array([cls_weights[lab] for lab in data['y_train']], dtype=np.float32)
    
    # 数据管道
    def create_dataset(X, y, w=None):
        ds = tf.data.Dataset.from_tensor_slices((X, y))
        if w is not None:
            # 正确的方式：将样本权重作为元组的第三个元素传递
            ds_w = tf.data.Dataset.from_tensor_slices(w)
            ds = tf.data.Dataset.zip((ds, ds_w))
            ds = ds.map(lambda xy, w: (aug_fn(xy[0]), xy[1], w), num_parallel_calls=tf.data.AUTOTUNE)
        else:
            ds = ds.map(lambda x, y: (aug_fn(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        return ds
    
    train_ds = create_dataset(data['X_train'], data['y_train'], sample_weights)
    train_ds = train_ds.shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE)
    
    val_ds = create_dataset(data['X_val'], data['y_val'])
    val_ds = val_ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    
    # 回调函数
    os.makedirs("models", exist_ok=True)
    ckpt = f"models/cldnn_classifier_{model_idx}.keras"
    callbacks = [
        EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=20, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=10),
        ModelCheckpoint(ckpt, save_best_only=True, monitor='val_sparse_categorical_accuracy', save_format="tf")
    ]
    
    print(f"\n🔥 训练CLDNN模型 {model_idx + 1}...")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    return model

# ---------- 主函数 ----------
def main():
    os.makedirs("models", exist_ok=True)
    os.makedirs("visualizations", exist_ok=True)
    
    print("=" * 80)
    print("🚀 开始训练CLDNN干扰分类模型")
    print("=" * 80)

    # 加载数据
    print("⏳ 加载数据集...")
    dataset = load_dataset("/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz")
    data = preprocess_data(dataset)
    
    # 训练参数
    n_models = 1  # 训练单个模型（如需集成可改为3）
    epochs = 120
    batch_size = 128

    # 训练模型
    model = train_single_model(data, model_idx=0, epochs=epochs, batch=batch_size)
    
    # 评估测试集
    test_loss, test_acc = model.evaluate(
        data['X_test'], data['y_test'],
        batch_size=batch_size, verbose=0
    )
    print(f"\n📊 测试集准确率: {test_acc:.4f}")
    
    # 保存最终模型
    model.save("models/cldnn_classifier_final.keras", save_format="tf")
    print("\n✅ 训练完成！模型已保存至 models/cldnn_classifier_final.keras")

if __name__ == "__main__":
    main()

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 开始训练CLDNN干扰分类模型
⏳ 加载数据集...


2025-10-21 23:25:20.777666: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-10-21 23:25:20.779228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-10-21 23:25:20.779960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


🔥 训练CLDNN模型 1...
Epoch 1/120


2025-10-21 23:25:21.339222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [56700]
	 [[{{node Placeholder/_3}}]]
2025-10-21 23:25:21.339439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [56700]
	 [[{{node Placeholder/_3}}]]
2025-10-21 23:25:21.526030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' 

443/443 [==============================] - ETA: 0s - loss: 1.3535 - sparse_categorical_accuracy: 0.4941

2025-10-21 23:25:37.950026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [12150]
	 [[{{node Placeholder/_1}}]]
2025-10-21 23:25:37.950227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [12150]
	 [[{{node Placeholder/_1}}]]
2025-10-21 23:25:38.214660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' 

443/443 [==============================] - 18s 30ms/step - loss: 1.3535 - sparse_categorical_accuracy: 0.4941 - val_loss: 1.8888 - val_sparse_categorical_accuracy: 0.4004 - lr: 0.0010
Epoch 2/120
443/443 [==============================] - 12s 28ms/step - loss: 0.9688 - sparse_categorical_accuracy: 0.6119 - val_loss: 0.8749 - val_sparse_categorical_accuracy: 0.6374 - lr: 0.0010
Epoch 3/120
443/443 [==============================] - 13s 28ms/step - loss: 0.8667 - sparse_categorical_accuracy: 0.6446 - val_loss: 0.8637 - val_sparse_categorical_accuracy: 0.6417 - lr: 0.0010
Epoch 4/120
443/443 [==============================] - 13s 28ms/step - loss: 0.8253 - sparse_categorical_accuracy: 0.6551 - val_loss: 0.7565 - val_sparse_categorical_accuracy: 0.6709 - lr: 0.0010
Epoch 5/120
443/443 [==============================] - 12s 28ms/step - loss: 0.8036 - sparse_categorical_accuracy: 0.6638 - val_loss: 0.8148 - val_sparse_categorical_accuracy: 0.6566 - lr: 0.0010
Epoch 6/120
443/443 [===========

In [5]:
# evaluate_cldnn_classifier.py
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                           classification_report)
from sklearn.preprocessing import StandardScaler
import matplotlib.font_manager as fm

# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载数据集并处理类型转换"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
        print("⚠️ 未找到干扰类型名称，使用默认值")
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
    else:
        type2label = {
            "satellite_signal": 0,
            "single_tone": 1,
            "comb_spectra": 2,
            "sweeping": 3,
            "pulse": 4,
            "frequency_hopping": 5,
            "noise_fm": 6,
            "noise_am": 7,
            "random_combination": 8
        }
    
    label2name = {int(i): str(type_names[k]) for k, i in type2label.items()}
    
    return {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "label2name": label2name,
        "L": int(data["L"])
    }

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据（仅分类任务）"""
    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]])

    return signals, dataset["labels"], dataset["jnr_values"], dataset["label2name"]

# ---------- 加载模型 ----------
def load_model(model_path):
    """加载训练好的CLDNN模型"""
    if os.path.exists(model_path):
        try:
            model = tf.keras.models.load_model(model_path)
            print(f"✅ 成功加载模型: {model_path}")
            return model
        except Exception as e:
            print(f"❌ 加载模型失败: {e}")
            return None
    print(f"⚠️ 未找到模型: {model_path}")
    return None

# ---------- 计算JNR准确率 ----------
def calculate_jnr_accuracy(y_true, y_pred, jnr_values, jnr_range):
    """计算每个JNR下的分类准确率"""
    jnr_acc = {}
    for jnr in jnr_range:
        mask = jnr_values == jnr
        if np.sum(mask) > 0:
            jnr_acc[jnr] = accuracy_score(y_true[mask], y_pred[mask])
        else:
            jnr_acc[jnr] = np.nan
    return jnr_acc

# ---------------------- 绘制混淆矩阵 ----------------------
def plot_confusion_matrix(cm, labels, title, xlabel, ylabel, filename, dpi=150, rotate_x=False):
    """专业混淆矩阵可视化"""
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_normalized = np.nan_to_num(cm_normalized)

    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(cm_normalized,
                     annot=True,
                     fmt='.2f',
                     cmap='Blues',
                     xticklabels=labels,
                     yticklabels=labels,
                     square=True,
                     annot_kws={"size": 14})
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)

    plt.title(title, pad=20, fontsize=18)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(rotation=45 if rotate_x else 0, ha='right' if rotate_x else 'center', fontsize=14)
    plt.yticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi)
    plt.close()
    
# ---------- 绘制JNR准确率曲线 ----------
def plot_jnr_accuracy(jnr_acc, filename):
    """绘制JNR-准确率曲线"""
    plt.figure(figsize=(10, 6))
    valid_jnr = [j for j in jnr_acc if not np.isnan(jnr_acc[j])]
    valid_acc = [jnr_acc[j] for j in valid_jnr]
    
    plt.plot(valid_jnr, valid_acc, 'o-', linewidth=2, markersize=8)
    plt.xlabel('JNR (dB)', fontsize=12)
    plt.ylabel('分类准确率', fontsize=12)
    plt.title('不同JNR下的干扰识别准确率', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(list(jnr_acc.keys()))
    plt.ylim(0, 1.05)
    plt.savefig(filename, dpi=300)
    plt.close()

# ---------- 主评估函数 ----------
def evaluate(model_path="models/cldnn_classifier_final.keras", 
             npz_path="/path/to/dataset.npz"):
    """主评估流程"""
    os.makedirs("visualizations", exist_ok=True)
    os.makedirs("reports", exist_ok=True)
    
    print("=" * 80)
    print("🚀 CLDNN干扰分类模型评估")
    print("=" * 80)

    # 1. 加载数据
    print("\n🔍 加载数据集...")
    dataset = load_dataset(npz_path)
    X_test, y_test, jnr_test, label2name = preprocess_data(dataset)
    
    # 2. 加载模型
    print("\n🔧 加载训练好的模型...")
    model = load_model(model_path)
    if not model:
        raise ValueError("❌ 没有可用的模型进行评估")
    
    # 3. 模型预测
    print("\n🔮 进行模型预测...")
    y_pred = np.argmax(model.predict(X_test, verbose=1), axis=1)
    
    # 4. 计算JNR准确率
    jnr_range = np.arange(-10, 31, 5)  # [-10, -5, 0, 5, ..., 30]
    jnr_acc = calculate_jnr_accuracy(y_test, y_pred, jnr_test, jnr_range)
    
    # 5. 生成混淆矩阵
    print("\n🎨 生成混淆矩阵...")
    cm = confusion_matrix(y_test, y_pred)
    class_names = [label2name[i] for i in sorted(label2name.keys())]
    plot_confusion_matrix(
        cm=cm,
        labels=class_names,
        title="CLDNN Classfication Confusion Matrix",
        xlabel="Predicted",
        ylabel="True",
        filename="visualizations/cldnn_confusion_matrix.png",
        dpi=300,
        rotate_x=True
    )
    
    # 6. 绘制JNR准确率曲线
    print("\n📈 生成JNR准确率曲线...")
    plot_jnr_accuracy(jnr_acc, "visualizations/cldnn_jnr_accuracy.png")
    
    # 7. 保存评估报告
    print("\n📝 生成评估报告...")
    report = {
        "overall_accuracy": float(accuracy_score(y_test, y_pred)),
        "jnr_accuracy": {int(j): float(acc) if not np.isnan(acc) else None 
                         for j, acc in jnr_acc.items()},
        "confusion_matrix": cm.tolist(),
        "class_names": class_names,
        "classification_report": classification_report(
            y_test, y_pred, 
            target_names=class_names,
            output_dict=True
        )
    }
    
    with open("reports/cldnn_evaluation_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4, ensure_ascii=False)
    
    # 8. 打印结果
    print("\n" + "="*50)
    print("📊 评估结果摘要")
    print("="*50)
    print(f"整体分类准确率: {report['overall_accuracy']:.4f}")
    
    print("\n各JNR下的分类准确率:")
    for jnr in sorted(jnr_acc.keys()):
        acc = jnr_acc[jnr]
        acc_str = f"{acc:.4f}" if not np.isnan(acc) else "N/A"
        print(f"  JNR={jnr}dB: {acc_str}")
    
    print("\n✅ 评估完成！结果已保存至 reports/cldnn_evaluation_report.json")

# ---------- 主程序 ----------
if __name__ == "__main__":
    evaluate(
        model_path="models/cldnn_classifier_final.keras",
        npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"
    )

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 CLDNN干扰分类模型评估

🔍 加载数据集...

🔧 加载训练好的模型...


2025-10-22 00:01:52.124582: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-10-22 00:01:52.125528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-10-22 00:01:52.126596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

✅ 成功加载模型: models/cldnn_classifier_final.keras

🔮 进行模型预测...


2025-10-22 00:01:52.858117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-10-22 00:01:52.859371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-10-22 00:01:52.860144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2532/2532 [==============================] - 22s 8ms/step

🎨 生成混淆矩阵...

📈 生成JNR准确率曲线...

📝 生成评估报告...

📊 评估结果摘要
整体分类准确率: 0.8864

各JNR下的分类准确率:
  JNR=-10dB: 0.6071
  JNR=-5dB: 0.7481
  JNR=0dB: 0.8433
  JNR=5dB: 0.9063
  JNR=10dB: 0.9464
  JNR=15dB: 0.9716
  JNR=20dB: 0.9837
  JNR=25dB: 0.9848
  JNR=30dB: 0.9862

✅ 评估完成！结果已保存至 reports/cldnn_evaluation_report.json


CLDNN_final_1021.ipynb评估结果摘要
==================================================
整体分类准确率: 0.8864

各JNR下的分类准确率:
  JNR=-10dB: 0.6071
  JNR=-5dB: 0.7481
  JNR=0dB: 0.8433
  JNR=5dB: 0.9063
  JNR=10dB: 0.9464
  JNR=15dB: 0.9716
  JNR=20dB: 0.9837
  JNR=25dB: 0.9848
  JNR=30dB: 0.9862